# Import software libraries

In [66]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [67]:
# Connect to SQLite database.
conn =sqlite3.connect('data/prod_sample.db')

conn

In [68]:
# List all the tables in the database.
query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""


users = pd.read_sql_query(query,conn)
users


,name
0,stock_description
1,online_retail_history


# Read data from the `online_retail_history` table

In [69]:
# Write the query to be executed that selects everything from the online_retail_history table.

online_retail_history = "select * from online_retail_history"


# Use the read_sql function in pandas to read a query into a DataFrame.

df = pd.read_sql(online_retail_history,conn)

# Preview the first five rows of the data.
df.head(5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [70]:
# Get the shape of the data.
df.shape


(15321, 8)

# Read data from the `stock_description` table

In [71]:
# Write the query to be executed that selects everything from the online_retail_history table.

stock_description = "select * from stock_description"



# Use the read_sql function in pandas to read a query into a DataFrame.

df = pd.read_sql(stock_description,conn)


# Preview the first five rows of the data.

df.head(5)

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [72]:
# Get the shape of the data.
df.shape


(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [73]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.



agg_data = "select * from online_retail_history left join stock_description on online_retail_history.stockcode =stock_description.stockcode"



# Use the read_sql function in pandas to read a query into a DataFrame.
df = pd.read_sql_query(agg_data,conn)


# Preview the first five rows of the data.
df.head(5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,84879,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT


In [74]:
# Get the shape of the data.

df.shape

(17032, 10)

# Identify and fix corrupt or unusable data

In [75]:
# Check the value counts of the "Description" field.
df.Description.count
# test = "select * from stock_description where description ='?'"
# df = pd.read_sql_query(test,conn)
# df


<bound method Series.count of 0        CREAM HANGING HEART T-LIGHT HOLDER
1             ASSORTED COLOUR BIRD ORNAMENT
2        CREAM HANGING HEART T-LIGHT HOLDER
3        CREAM HANGING HEART T-LIGHT HOLDER
4                   LUNCH BAG RED RETROSPOT
                        ...                
17027               LUNCH BAG  BLACK SKULL.
17028               LUNCH BAG RED RETROSPOT
17029                        POPCORN HOLDER
17030               LUNCH BAG RED RETROSPOT
17031         ASSORTED COLOUR BIRD ORNAMENT
Name: Description, Length: 17032, dtype: object>

In [76]:
# Remove rows where "Description" is just a question mark (?).


cleaned_description = "select * from online_retail_history left join stock_description on online_retail_history.stockcode =stock_description.stockcode where description != '?'"


df = pd.read_sql_query(cleaned_description,conn)

# Preview the first five rows of the data.
df


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT
1,536401,20725,1,2010-12-01 11:21:00,1.65,u15862,United Kingdom,1.65,20725,LUNCH BAG RED RETROSPOT
2,536412,20725,1,2010-12-01 11:49:00,1.65,u1792,United Kingdom,1.65,20725,LUNCH BAG RED RETROSPOT
3,536544,20725,4,2010-12-01 14:32:00,4.21,None,United Kingdom,16.84,20725,LUNCH BAG RED RETROSPOT
4,536577,20725,60,2010-12-01 16:13:00,1.45,u13777,United Kingdom,87.00,20725,LUNCH BAG RED RETROSPOT
...,...,...,...,...,...,...,...,...,...,...
15316,581439,85123A,1,2011-12-08 16:30:00,5.79,None,United Kingdom,5.79,85123A,CREAM HANGING HEART T-LIGHT HOLDER
15317,581452,85123A,32,2011-12-08 18:03:00,2.55,u17675,United Kingdom,81.60,85123A,CREAM HANGING HEART T-LIGHT HOLDER
15318,581472,85123A,6,2011-12-08 19:55:00,2.95,u15796,United Kingdom,17.70,85123A,CREAM HANGING HEART T-LIGHT HOLDER
15319,581492,85123A,3,2011-12-09 10:03:00,5.79,None,United Kingdom,17.37,85123A,CREAM HANGING HEART T-LIGHT HOLDER


# Identify and remove duplicates

In [77]:
# Identify all duplicated data.

df.duplicated().sum()



115

In [78]:
# Print the duplicated data.
dupli = df[df.duplicated(keep=False)]
print(dupli)


      Invoice StockCode  Quantity          InvoiceDate  Price CustomerID  \
40     537955     20725         1  2010-12-09 11:28:00   1.65     u16782   
41     537955     20725         1  2010-12-09 11:28:00   1.65     u16782   
281    546635     20725         1  2011-03-15 12:11:00   1.65     u16392   
282    546635     20725         1  2011-03-15 12:11:00   1.65     u16392   
605    555560     20725         1  2011-06-05 14:20:00   1.65     u17346   
...       ...       ...       ...                  ...    ...        ...   
15010  573543    85123A         2  2011-10-31 13:05:00   2.95     u17777   
15196  578255    85123A         1  2011-11-23 12:58:00   2.95     u14675   
15197  578255    85123A         1  2011-11-23 12:58:00   2.95     u14675   
15241  579458    85123A         1  2011-11-29 14:38:00   2.95     u16549   
15242  579458    85123A         1  2011-11-29 14:38:00   2.95     u16549   

              Country  TotalAmount StockCode  \
40     United Kingdom         1.65     

In [79]:
# Remove the duplicated data.
duplicate_removed = df.drop_duplicates()






# Preview the first five rows of the data.
duplicate_removed.head(5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT
1,536401,20725,1,2010-12-01 11:21:00,1.65,u15862,United Kingdom,1.65,20725,LUNCH BAG RED RETROSPOT
2,536412,20725,1,2010-12-01 11:49:00,1.65,u1792,United Kingdom,1.65,20725,LUNCH BAG RED RETROSPOT
3,536544,20725,4,2010-12-01 14:32:00,4.21,None,United Kingdom,16.84,20725,LUNCH BAG RED RETROSPOT
4,536577,20725,60,2010-12-01 16:13:00,1.45,u13777,United Kingdom,87.00,20725,LUNCH BAG RED RETROSPOT


# Correct date formats

In [80]:
# Get the data types for every column in the DataFrame.

duplicate_removed.dtypes

Invoice         object
StockCode       object
Quantity         int64
InvoiceDate     object
Price          float64
CustomerID      object
Country         object
TotalAmount    float64
StockCode       object
Description     object
dtype: object

In [81]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format="%Y-%m-%d")


In [82]:
# Get the data types for every column in the converted DataFrame.
df.dtypes

Invoice                object
StockCode              object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
CustomerID             object
Country                object
TotalAmount           float64
StockCode              object
Description            object
dtype: object

# Examine the table before finishing

In [83]:
# Preview the first five rows of the data.
df.head(5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,StockCode,Description
0,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,20725,LUNCH BAG RED RETROSPOT
1,536401,20725,1,2010-12-01 11:21:00,1.65,u15862,United Kingdom,1.65,20725,LUNCH BAG RED RETROSPOT
2,536412,20725,1,2010-12-01 11:49:00,1.65,u1792,United Kingdom,1.65,20725,LUNCH BAG RED RETROSPOT
3,536544,20725,4,2010-12-01 14:32:00,4.21,None,United Kingdom,16.84,20725,LUNCH BAG RED RETROSPOT
4,536577,20725,60,2010-12-01 16:13:00,1.45,u13777,United Kingdom,87.00,20725,LUNCH BAG RED RETROSPOT


# Load the dataset into a pickle file

In [84]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.
df.to_pickle("online_history_cleaned.pickle")


In [85]:
# Close any connections to the database.
conn.close()
